<a href="https://colab.research.google.com/github/linlcc/NLP/blob/master/1_Sentiment_Classificaiton.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this experiment, I will train models to distringuish examples of two different genres of Shakespeare's plays: comedies and tragedies. (We'll ignore the histories, sonnets, etc.) Since he died four hundred years ago, Shakespeare has not written any more plays—although scraps of various other works have come to light. We are not, therefore, interested in building models simply to help categorize an unbounded stream of future documents, as we might be in other applications of text classification; rather, we are interested in what a classifier might have to tell us about what we mean by the terms “comedy” and “tragedy”.

I will start by copying and running your `createBasicFeatures` function from the experiment with movie reviews. Do the features the classifier focuses on tell you much about comedy and tragedy in general?

I will then implement another featurization function `createInterestingFeatures`, which will focus on only those features you think are informative for distinguishing between comedy and tragedy. Accuracy on leave-one-out cross-validation may go up, but it more important to look at the features given the highest weight by the classifier. Interpretability in machine learning, of course, may be harder to define than accuracy—although accuracy at some tasks such as summarization is hard enoough.

In [0]:
import json
import requests
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import cross_validate,LeaveOneOut,KFold
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

In [0]:
#read in the shakespeare corpus
def readShakespeare():
  raw = requests.get("https://raw.githubusercontent.com/mutherr/CS6120-PS1-data/master/shakespeare_plays.json").text.strip()
  corpus = [json.loads(line) for line in raw.split("\n")]

  #remove histories from the data, as we're only working with tragedies and comedies
  corpus = [entry for entry in corpus if entry["genre"] != "history"]
  return corpus

In [3]:
!pip install vaderSentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer 
def sentiment_scores(sentence): 
  
    sid_obj = SentimentIntensityAnalyzer() 
    sentiment_dict = sid_obj.polarity_scores(sentence) 

    return sentiment_dict['compound']

Implement two functions to featurize the data:

In [0]:
#NB: The current contents are for testing only
#This function should return: 
#  -a sparse numpy matrix of document features
#  -a list of the correct genre for each document
#  -a list of the vocabulary used by the features, such that the ith term of the
#    list is the word whose counts appear in the ith column of the matrix. 

# create a feature representation using all tokens that contain an alphabetic character.

def createBasicFeatures(corpus):
    con=[]
    classes=[]
    for item in corpus:
        con.append(list(item.values())[2]) # put all contens in a list
        classes.append(list(item.values())[0]) # list of the correst classes

    count_vect = CountVectorizer()
    texts = count_vect.fit_transform(con) # creat sparse matrix of contect list
    vocab=count_vect.get_feature_names() # list of the vocabularies
    return texts,classes,vocab


def createInterestingFeatures(corpus):
    con=[]
    classes=[]
    for item in corpus:
        con.append(list(item.values())[2]) # put all contens in a list
        classes.append(list(item.values())[0]) # list of the correst classes
        
        
    count_vect = CountVectorizer(ngram_range=(1, 2),
                                 stop_words='english',token_pattern=r'\b[^\d\W]+\b',max_df=0.9,max_features = 2000)
    count_vect.fit_transform(con)
    texts = count_vect.transform(con)
    vocab=count_vect.get_feature_names()

# get the words which contains emotions
    sei=[] 
    for item in vocab:
        sei.append(sentiment_scores(item))

# multiply counts with emotion scores
    for i in range(texts.shape[0]):
        for j in range(texts.shape[1]):
            texts[i,j]=texts[i,j]*sei[j]

            
    return texts,classes,vocab

In [0]:
#given a numpy matrix representation of the features for the training set, the 
# vector of true classes for each example, and the vocabulary as described 
# above, this computes the accuracy of the model using leave one out cross 
# validation and reports the most indicative features for each class
def evaluateModel(X,y,vocab,penalty="l1"):
  #create and fit the model
  model = LogisticRegression(penalty=penalty,solver="liblinear")
  results = cross_validate(model,X,y,cv=LeaveOneOut())
  
  #determine the average accuracy
  scores = results["test_score"]
  avg_score = sum(scores)/len(scores)
  
  #determine the most informative features
  # this requires us to fit the model to everything, because we need a
  # single model to draw coefficients from, rather than 26
  model.fit(X,y)
  neg_class_prob_sorted = model.coef_[0, :].argsort()
  pos_class_prob_sorted = (-model.coef_[0, :]).argsort()
#   print(pos_class_prob_sorted,vocab)
  termsToTake = 20
  pos_indicators = [vocab[i] for i in neg_class_prob_sorted[:termsToTake]]

  neg_indicators = [vocab[i] for i in pos_class_prob_sorted[:termsToTake]]

  return avg_score,pos_indicators,neg_indicators

def runEvaluation(X,y,vocab):
  print("----------L1 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l1")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)
  #this call will fit a model with L2 normalization
  print("----------L2 Norm-----------")
  avg_score,pos_indicators,neg_indicators = evaluateModel(X,y,vocab,"l2")
  print("The model's average accuracy is %f"%avg_score)
  print("The most informative terms for pos are: %s"%pos_indicators)
  print("The most informative terms for neg are: %s"%neg_indicators)

In [0]:
corpus = readShakespeare()

Run the following to train and evaluate two models with basic features:

In [7]:
X,y,vocab = createBasicFeatures(corpus)
runEvaluation(X, y, vocab)

----------L1 Norm-----------
The model's average accuracy is 0.769231
The most informative terms for pos are: ['helena', 'prospero', 'sir', 'you', 'your', 'me', 'for', 'duke', 'of', 'love', 'preserver', 'preserved', 'preserve', 'preserv', 'preservation', 'preservers', 'presents', 'presentment', 'presently', 'presenting']
The most informative terms for neg are: ['our', 'him', 'rom', 'iago', 'thy', 'ham', 'imogen', 'what', 'brutus', 'his', 'lear', 'timon', 'preservers', 'preserver', 'preserved', 'preserv', 'preservative', 'preservation', 'presents', 'presentment']
----------L2 Norm-----------
The model's average accuracy is 0.730769
The most informative terms for pos are: ['you', 'prospero', 'duke', 'helena', 'antonio', 'me', 'for', 'your', 'sir', 'ariel', 'sebastian', 'hermia', 'lysander', 'parolles', 'stephano', 'will', 'leontes', 'caliban', 'demetrius', 'love']
The most informative terms for neg are: ['ham', 'iago', 'him', 'our', 'othello', 'what', 'his', 'lear', 'imogen', 'brutus', '

Run the following to train and evaluate two models with features that are interesting for distinguishing comedy and tragedy:

In [8]:
X,y,vocab = createInterestingFeatures(corpus)
runEvaluation(X, y, vocab)

/usr/local/lib/python3.6/dist-packages/scipy/sparse/_index.py:84: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_intXint(row, col, x.flat[0])


----------L1 Norm-----------
The model's average accuracy is 0.884615
The most informative terms for pos are: ['poison', 'whore', 'war', 'revenge', 'lover', 'bloody', 'honour', 'sick', 'humour', 'beauty', 'hero', 'profit', 'praise', 'prais d', 'prais', 'practice', 'powers', 'power', 'pow r', 'pow']
The most informative terms for neg are: ['ass', 'drown', 'good night', 'shylock', 'good lord', 'pray sir', 'pray let', 'praise', 'prais d', 'pray thee', 'practice', 'prayer', 'powers', 'power', 'pow r', 'pow', 'prais', 'pound', 'posthumus', 'post']
----------L2 Norm-----------
The model's average accuracy is 0.923077
The most informative terms for pos are: ['war', 'poison', 'sick', 'whore', 'honour', 'revenge', 'lover', 'bloody', 'humour', 'beauty', 'good sir', 'woo', 's dead', 'favour', 'grave', 'villains', 'true love', 'lovers', 'ghost', 'worst']
The most informative terms for neg are: ['ass', 'good night', 'drown', 'burden', 'drown d', 'pains', 'precious', 'scorn', 'proud', 'melancholy', 